In [1]:
# Imports
import math
import nltk
import re
import inflect
import json
import numpy as np
import pandas as pd
from nltk.tag import SequentialBackoffTagger
from nltk.tokenize import word_tokenize, sent_tokenize, PunktSentenceTokenizer, RegexpTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords, state_union
from nltk.wsd import lesk
from nltk.tag import UnigramTagger, BigramTagger, BrillTagger, brill, BrillTaggerTrainer
from nltk.chunk import ne_chunk

In [2]:
matches = pd.read_csv('matches.csv')
deliveries = pd.read_csv('deliveries.csv')

In [3]:
class Executors:
    
    def __init__(self):
        self.matches = pd.read_csv("matches.csv")
        self.deliveries = pd.read_csv('deliveries.csv')
    
    def runs_batsman_match(self,batsman_name, match_id):
        x = self.deliveries.groupby(['match_id', 'batsman'])['batsman_runs'].sum()
        runs = x[match_id][batsman_name]
        return runs 



    ## Total Runs scored in Match X by team Y
    def total_runs_team_match(self,team, match_id):
        x = self.deliveries.groupby(['match_id','batting_team'])['total_runs'].sum()
        total_runs_match = x[match_id][team]
        return total_runs_match  


    # In[5]:

    ## Max scorer in match X
    def max_score_batsman_match(self,match_id):
        x = self.deliveries.groupby(['match_id', 'batsman'])['batsman_runs'].sum()
        name = x[match_id].idxmax()
        runs = x[match_id].max()
        return [name, runs]  


    # In[6]:

    ## Min scorer in match X
    def min_score_batsman_match(self,match_id):
        x = self.deliveries.groupby(['match_id', 'batsman'])['batsman_runs'].sum()
        name = x[match_id].idxmin()
        runs = x[match_id].min()
        return [name, runs]  


    # In[7]:

    ## Maxscorer in match X in Team Y
    def max_score_batsman_match_inTeam(self,match_id, team):
        x = self.deliveries.groupby(['match_id','batting_team', 'batsman'])['batsman_runs'].sum()
        name = x[match_id][team].idxmax()
        runs = x[match_id][team].max()
        return [name, runs]  


    # In[8]:

    ## Min scorer in match X in Team Y
    def min_score_batsman_match_inTeam(self,match_id, team):
        x = self.deliveries.groupby(['match_id','batting_team', 'batsman'])['batsman_runs'].sum()
        name = x[match_id][team].idxmin()
        runs = x[match_id][team].min()
        return [name, runs]  


    # In[9]:

    ## ith highest scorer 
    def highest_scorer(self,i):
        #player_runs = self.deliveries.groupby('batsman')['batsman_runs'].sum().sort_values(ascending =False)
        player_name = self.deliveries.groupby('batsman')['batsman_runs'].sum().sort_values(ascending =False).iloc[i:i+1]
        return  player_name


    # In[10]:

    ## batsman total run
    def total_runs_batsman_IPL(self,batsman):
        x = self.deliveries.groupby(['batsman'])['batsman_runs'].sum()
        runs_batsman = x[batsman]
        return runs_batsman


    # In[11]:

    #Total runsmade by a team
    def total_runs_team_IPL(self,team):
        x = self.deliveries.groupby(['batting_team'])['total_runs'].sum()
        total_runs_IPL = x[team]
        return total_runs_IPL


    # ## Boundary

    # In[12]:

    def dot_balls_batsman_match(self,batsman, match_id):
        x = self.deliveries[(self.deliveries['batsman'] == batsman) & (self.deliveries['match_id'] == match_id) & (self.deliveries['total_runs'] == 0)]
        dot_balls = x.shape[0]
        return dot_balls


    # In[13]:

    #boundaries hit by a particular batman in particular match
    def b_4_batsman_match(self,batsman, match_id):
        x = self.deliveries[(self.deliveries['batsman'] == batsman) & (self.deliveries['match_id'] == match_id) & (self.deliveries['total_runs'] == 4)]
        b_4 = x.shape[0]
        return b_4


    # In[14]:


    #Sixes hit by batsman inparticular match
    def b_6_batsman_match(self,batsman, match_id):
        x = self.deliveries[(self.deliveries['batsman'] == batsman) & (self.deliveries['match_id'] == match_id) & (self.deliveries['total_runs'] == 6)]
        b_6 = x.shape[0]
        return b_6


    # In[15]:

    def team_fours(self,match_id, batting_team):
        team_fours = self.deliveries[self.deliveries.batsman_runs == 4]
        team_fours_count = team_fours.groupby(['match_id','batting_team']).count()['inning']
        return team_fours_count[match_id][batting_team]

    def team_sixes(self,match_id, batting_team):
        team_sixes = self.deliveries[self.deliveries.batsman_runs == 6]
        team_sixes_count = team_sixes.groupby(['match_id','batting_team']).count()['inning']
        return team_sixes_count[match_id][batting_team]


    def overall_fours_count(self,batsman):
        fours = self.deliveries[self.deliveries.batsman_runs == 4]
        fours_count = fours.groupby('batsman').count()['inning']
        return fours_count[batsman]


    def overall_sixes_count(self,batsman):
        sixes = self.deliveries[self.deliveries.batsman_runs == 6]
        sixes_count = sixes.groupby('batsman').count()['inning']
        return sixes_count[batsman]


    def most_fours_count(self, i):
        fours = self.deliveries[self.deliveries.batsman_runs == 4]
        fours_count = fours.groupby('batsman').count()['inning']
        return fours_count.sort_values(ascending = False).iloc[i:i+1]


    def most_sixes_count(self):
        sixes = self.deliveries[self.deliveries.batsman_runs == 6]
        sixes_count = sixes.groupby('batsman').count()['inning'].sort_values(ascending = False).iloc[0:1]
        return sixes_count


    # ## Strike Rate

    # In[16]:

    def balls_faced_batsman_match(self,batsman, match_id):
        x = self.deliveries[(self.deliveries['batsman'] == batsman) & (self.deliveries['match_id'] == match_id) & (self.deliveries['wide_runs'] == 0)]
        return x.shape[0]
    def strikeRate_batsman_match(self,batsman, match_id):
        runs = runs_batsman_match(batsman, match_id)
        balls = balls_faced_batsman_match(batsman, match_id)

        strike_rate = runs/balls * 100
        return strike_rate

    def total_runs_scored_IPL(self):
        runs_count = self.deliveries.groupby('batsman')['batsman_runs'].sum()
        return runs_count

    def total_ball_faced_IPL(self):
        balls = self.deliveries[(self.deliveries.wide_runs == 0)].groupby('batsman')['inning']
        balls_count = balls.count()
        return balls_count

    def total_strike_rate_IPL(self,i):
        strike_rate = (total_runs_scored_IPL()/total_ball_faced_IPL())*100
        return strike_rate.sort_values(ascending = False).iloc[i:i+1]
    
    def overall_economy_rate_by_bowler(self, match_id=0, team=None, bowler=None):
        bowler_eco = []
        if(match_id==0):
            if(team is not None):
                runs_conceded = self.deliveries[self.deliveries['bowling_team'] == team].total_runs.sum()
                balls = (self.deliveries[(self.deliveries['bowling_team'] == team) & (self.deliveries['wide_runs'] == 0) & (self.deliveries['is_super_over'] == 0) & (self.deliveries['noball_runs']==0)]).ball.count()
                dot_balls = (self.deliveries[(self.deliveries['bowling_team'] == team) & (self.deliveries['wide_runs'] == 0) & (self.deliveries['is_super_over'] == 0) & (self.deliveries['noball_runs']==0)& (self.deliveries['total_runs']==0)]).ball.count()
                overs = float(int(balls/6) + float(balls%6)/10)
                frac, whole = math.modf(overs)
                total = whole + frac*10/6
                economy_rate = runs_conceded/total
                bowler_eco.append((team, economy_rate, balls, overs, dot_balls))
            elif (bowler == None): 
                bowlers = self.deliveries.bowler.unique()
                for bowler in bowlers:
                    runs_conceded = self.deliveries[self.deliveries['bowler'] == bowler].total_runs.sum()-deliveries[self.deliveries['bowler'] == bowler].bye_runs.sum()-deliveries[self.deliveries['bowler'] == bowler].legbye_runs.sum() 
                    balls = (self.deliveries[(self.deliveries['bowler']== bowler) & (self.deliveries['wide_runs'] == 0) & (self.deliveries['is_super_over'] == 0) & (self.deliveries['noball_runs']==0)]).ball.count()
                    dot_balls = (self.deliveries[(self.deliveries['bowler']== bowler) & (self.deliveries['wide_runs'] == 0) & (self.deliveries['is_super_over'] == 0) & (self.deliveries['noball_runs']==0)& (self.deliveries['total_runs']==0)]).ball.count()
                    overs = float(int(balls/6) + float(balls%6)/10)
                    frac, whole = math.modf(overs)
                    total = whole + frac*10/6
                    economy_rate = runs_conceded/total
                    bowler_eco.append((bowler, economy_rate, balls, overs, dot_balls))
            else:
                runs_conceded = self.deliveries[self.deliveries['bowler'] == bowler].total_runs.sum()-deliveries[self.deliveries['bowler'] == bowler].bye_runs.sum()-deliveries[self.deliveries['bowler'] == bowler].legbye_runs.sum()
                balls = (self.deliveries[(self.deliveries['bowler']== bowler) & (self.deliveries['wide_runs'] == 0) & (self.deliveries['is_super_over'] == 0) & (self.deliveries['noball_runs']==0)]).ball.count()
                dot_balls = (self.deliveries[(self.deliveries['bowler']== bowler) & (self.deliveries['wide_runs'] == 0) & (self.deliveries['is_super_over'] == 0) & (self.deliveries['noball_runs']==0)& (self.deliveries['total_runs']==0)]).ball.count()
                overs = float(int(balls/6) + float(balls%6)/10)
                frac, whole = math.modf(overs)
                total = whole + frac*10/6
                economy_rate = runs_conceded/total
                bowler_eco.append((bowler, economy_rate, balls, overs, dot_balls))
        else:
            if (team is not None):
                runs_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowling_team'] == team)].total_runs.sum()
                balls = (self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowling_team'] == team) & (self.deliveries['wide_runs'] == 0) & (self.deliveries['is_super_over'] == 0) & (self.deliveries['noball_runs']==0)]).ball.count()
                dot_balls = (self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowling_team'] == team) & (self.deliveries['wide_runs'] == 0) & (self.deliveries['is_super_over'] == 0) & (self.deliveries['noball_runs']==0)& (self.deliveries['total_runs']==0)]).ball.count()
                overs = float(int(balls/6) + float(balls%6)/10)
                frac, whole = math.modf(overs)
                total = whole + frac*10/6
                economy_rate = runs_conceded/total
                bowler_eco.append((team, economy_rate, balls, overs, dot_balls))
            elif (bowler == None): 
                bowlers = self.deliveries[self.deliveries['match_id'] == match_id].bowler.unique()
                for bowler in bowlers:
                    runs_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].total_runs.sum()-deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].bye_runs.sum()-deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].legbye_runs.sum()
                    balls = (self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler']== bowler) & (self.deliveries['wide_runs'] == 0) & (self.deliveries['is_super_over'] == 0) & (self.deliveries['noball_runs']==0)]).ball.count()
                    dot_balls = (self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler']== bowler) & (self.deliveries['wide_runs'] == 0) & (self.deliveries['is_super_over'] == 0) & (self.deliveries['noball_runs']==0)& (self.deliveries['total_runs']==0)]).ball.count()
                    overs = float(int(balls/6) + float(balls%6)/10)
                    frac, whole = math.modf(overs)
                    total = whole + frac*10/6
                    economy_rate = runs_conceded/total
                    bowler_eco.append((bowler, economy_rate, balls, overs, dot_balls))
            else:
                runs_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].total_runs.sum()-deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].bye_runs.sum()-deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].legbye_runs.sum()
                balls = (self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler']== bowler) & (self.deliveries['wide_runs'] == 0) & (self.deliveries['is_super_over'] == 0) & (self.deliveries['noball_runs']==0)]).ball.count()
                dot_balls = (self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler']== bowler) & (self.deliveries['wide_runs'] == 0) & (self.deliveries['is_super_over'] == 0) & (self.deliveries['noball_runs']==0)& (self.deliveries['total_runs']==0)]).ball.count()
                overs = float(int(balls/6) + float(balls%6)/10)
                frac, whole = math.modf(overs)
                total = whole + frac*10/6
                economy_rate = runs_conceded/total
                bowler_eco.append((bowler, economy_rate, balls, overs, dot_balls))

        return  bowler_eco
    
    def bowler_balls(self, match_id=0, team = None, bowler=None, economy_rate=None, balls=None, overs=None,dot_balls=None, rank=1):
        eco_balls_over = overall_economy_rate_by_bowler(match_id=match_id,team=team, bowler=bowler)
        if(economy_rate is not None):
            eco_balls_over = sorted(eco_balls_over, key=lambda x: x[1])
            return eco_balls_over[rank-1][0],eco_balls_over[rank-1][1] 
        elif(balls is not None):
            eco_balls_over = sorted(eco_balls_over, key=lambda x: x[2], reverse=True)
            return eco_balls_over[rank-1][0],eco_balls_over[rank-1][2]
        elif(overs is not None):
            eco_balls_over = sorted(eco_balls_over, key=lambda x: x[3], reverse=True)
            return eco_balls_over[rank-1][0],eco_balls_over[rank-1][3]
        elif(dot_balls is not None):
            eco_balls_over = sorted(eco_balls_over, key=lambda x: x[4], reverse=True)
            return eco_balls_over[rank-1][0],eco_balls_over[rank-1][4]
        else:
            eco_balls_over = sorted(eco_balls_over, key=lambda x: x[1])
            return eco_balls_over[rank-1]
        
    def overall_runs_conceded(self, match_id=0,team=None, bowler=None):
        over_all_runs = []
        if(match_id==0):
            if(team is not None):
                total_runs_conceded = self.deliveries[self.deliveries['bowling_team'] == team].total_runs.sum()
                bye_runs_conceded = self.deliveries[self.deliveries['bowling_team'] == team].bye_runs.sum()
                legbye_runs_conceded = self.deliveries[self.deliveries['bowling_team'] == team].legbye_runs.sum()
                wide = self.deliveries[self.deliveries['bowling_team'] == team].wide_runs.sum()
                noball = self.deliveries[self.deliveries['bowling_team'] == team].noball_runs.sum()
                four_boundary_conceded = self.deliveries[(self.deliveries['bowling_team'] == team)&(self.deliveries['batsman_runs']==4)].batsman_runs.count()
                six_boundary_conceded = self.deliveries[(self.deliveries['bowling_team'] == team)&(self.deliveries['batsman_runs']==6)].batsman_runs.count()
                runs = total_runs_conceded
                total_boundaries = four_boundary_conceded + six_boundary_conceded
                extras = wide + noball + bye_runs_conceded + legbye_runs_conceded
                over_all_runs.append((team, runs ,wide, noball,extras, four_boundary_conceded, six_boundary_conceded, total_boundaries))          
            elif (bowler == None):
                bowlers = self.deliveries.bowler.unique()
                for bowler in bowlers:
                    total_runs_conceded = self.deliveries[self.deliveries['bowler'] == bowler].total_runs.sum()
                    bye_runs_conceded = self.deliveries[self.deliveries['bowler'] == bowler].bye_runs.sum()
                    legbye_runs_conceded = self.deliveries[self.deliveries['bowler'] == bowler].legbye_runs.sum()
                    wide = self.deliveries[self.deliveries['bowler']== bowler].wide_runs.sum()
                    noball = self.deliveries[self.deliveries['bowler']== bowler].noball_runs.sum()
                    four_boundary_conceded = self.deliveries[(self.deliveries['bowler'] == bowler)&(self.deliveries['batsman_runs']==4)].batsman_runs.count()
                    six_boundary_conceded = self.deliveries[(self.deliveries['bowler'] == bowler)&(self.deliveries['batsman_runs']==6)].batsman_runs.count()
                    runs = total_runs_conceded - bye_runs_conceded - legbye_runs_conceded
                    total_boundaries = four_boundary_conceded + six_boundary_conceded
                    extras = wide + noball + bye_runs_conceded + legbye_runs_conceded
                    over_all_runs.append((bowler, runs ,wide, noball,extras, four_boundary_conceded, six_boundary_conceded, total_boundaries))


            else:
                total_runs_conceded = self.deliveries[self.deliveries['bowler'] == bowler].total_runs.sum()
                wide = self.deliveries[self.deliveries['bowler']== bowler].wide_runs.sum()
                noball = self.deliveries[self.deliveries['bowler']== bowler].noball_runs.sum()    
                bye_runs_conceded = self.deliveries[self.deliveries['bowler'] == bowler].bye_runs.sum()
                legbye_runs_conceded = self.deliveries[self.deliveries['bowler'] == bowler].legbye_runs.sum()
                four_boundary_conceded = self.deliveries[(self.deliveries['bowler'] == bowler)&(self.deliveries['batsman_runs']==4)].batsman_runs.count()
                six_boundary_conceded = self.deliveries[(self.deliveries['bowler'] == bowler)&(self.deliveries['batsman_runs']==6)].batsman_runs.count()
                runs = total_runs_conceded - bye_runs_conceded - legbye_runs_conceded
                total_boundaries = four_boundary_conceded + six_boundary_conceded
                extras = wide + noball + bye_runs_conceded + legbye_runs_conceded
                over_all_runs.append((bowler, runs ,wide, noball,extras, four_boundary_conceded, six_boundary_conceded, total_boundaries))

        else:
            if(team is not None):
                total_runs_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowling_team'] == team)].total_runs.sum()
                wide = self.deliveries[(self.deliveries['match_id']==match_id)& (self.deliveries['bowling_team'] == team)].wide_runs.sum()
                noball = self.deliveries[(self.deliveries['match_id']==match_id)& (self.deliveries['bowling_team'] == team)].noball_runs.sum()    
                bye_runs_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowling_team'] == team)].bye_runs.sum()
                legbye_runs_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowling_team'] == team)].legbye_runs.sum()
                four_boundary_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowling_team'] == team)&(self.deliveries['batsman_runs']==4)].batsman_runs.count()
                six_boundary_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowling_team'] == team)&(self.deliveries['batsman_runs']==6)].batsman_runs.count()
                runs = total_runs_conceded
                total_boundaries = four_boundary_conceded + six_boundary_conceded
                extras = wide + noball + bye_runs_conceded + legbye_runs_conceded
                over_all_runs.append((team, runs ,wide, noball,extras, four_boundary_conceded, six_boundary_conceded, total_boundaries))

            elif(bowler == None):
                bowlers = self.deliveries[self.deliveries['match_id'] == match_id].bowler.unique()
                for bowler in bowlers:
                    total_runs_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].total_runs.sum()
                    bye_runs_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].bye_runs.sum()
                    legbye_runs_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].legbye_runs.sum()
                    wide = self.deliveries[(self.deliveries['match_id']==match_id)& (self.deliveries['bowler']== bowler)].wide_runs.sum()
                    noball = self.deliveries[(self.deliveries['match_id']==match_id)& (self.deliveries['bowler']== bowler)].noball_runs.sum()
                    four_boundary_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)&(self.deliveries['batsman_runs']==4)].batsman_runs.count()
                    six_boundary_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)&(self.deliveries['batsman_runs']==6)].batsman_runs.count()
                    runs = total_runs_conceded - bye_runs_conceded - legbye_runs_conceded
                    total_boundaries = four_boundary_conceded + six_boundary_conceded
                    extras = wide + noball + bye_runs_conceded + legbye_runs_conceded
                    over_all_runs.append((bowler, runs ,wide, noball,extras, four_boundary_conceded, six_boundary_conceded, total_boundaries))
            else:
                total_runs_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].total_runs.sum()
                wide = self.deliveries[(self.deliveries['match_id']==match_id)& (self.deliveries['bowler']== bowler)].wide_runs.sum()
                noball = self.deliveries[(self.deliveries['match_id']==match_id)& (self.deliveries['bowler']== bowler)].noball_runs.sum()    
                bye_runs_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].bye_runs.sum()
                legbye_runs_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].legbye_runs.sum()
                four_boundary_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)&(self.deliveries['batsman_runs']==4)].batsman_runs.count()
                six_boundary_conceded = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)&(self.deliveries['batsman_runs']==6)].batsman_runs.count()
                runs = total_runs_conceded - bye_runs_conceded - legbye_runs_conceded
                total_boundaries = four_boundary_conceded + six_boundary_conceded
                extras = wide + noball + bye_runs_conceded + legbye_runs_conceded
                over_all_runs.append((bowler, runs ,wide, noball,extras, four_boundary_conceded, six_boundary_conceded, total_boundaries))
        return over_all_runs
    
    def bowler_runs(self, match_id=0, team=None, bowler=None, runs=None, wide=None, noball=None,extras=None, fours=None, sixes=None, boundary=None, rank=1):
        bowler_stats_data = overall_runs_conceded(match_id=match_id,team=team, bowler=bowler)
        if (runs is not None):
            bowler_stats_data = sorted(bowler_stats_data, key=lambda x: x[1], reverse=True)
            return bowler_stats_data[rank-1][0:2]
        elif (wide is not None):
            bowler_stats_data = sorted(bowler_stats_data, key=lambda x: x[2], reverse=True)
            return bowler_stats_data[rank-1][0],bowler_stats_data[rank-1][2]
        elif (noball is not None):
            bowler_stats_data = sorted(bowler_stats_data, key=lambda x: x[3], reverse=True)
            return bowler_stats_data[rank-1][0],bowler_stats_data[rank-1][3]
        elif (extras is not None):
            bowler_stats_data = sorted(bowler_stats_data, key=lambda x: x[4], reverse=True)
            return bowler_stats_data[rank-1][0],bowler_stats_data[rank-1][4]
        elif (fours is not None):
            bowler_stats_data = sorted(bowler_stats_data, key=lambda x: x[5], reverse=True)
            return bowler_stats_data[rank-1][0],bowler_stats_data[rank-1][5]
        elif (sixes is not None):
            bowler_stats_data = sorted(bowler_stats_data, key=lambda x: x[6], reverse=True)
            return bowler_stats_data[rank-1][0],bowler_stats_data[rank-1][6]
        elif (boundary is not None):
            bowler_stats_data = sorted(bowler_stats_data, key=lambda x: x[7], reverse=True)
            return bowler_stats_data[rank-1][0],bowler_stats_data[rank-1][7]
        else:
            bowler_stats_data = sorted(bowler_stats_data, key=lambda x: x[1], reverse=True)
            return bowler_stats_data[rank-1][0:2]
        
        def bowler_wickets(sematch_id=0, team=None, bowler=None,rank =1):
            bowler_wicket =[]
            if(match_id==0):
                if (team is not None):
                    total_wickets = self.deliveries[self.deliveries['bowling_team'] == team].dismissal_kind.count()
                    wickets_total = total_wickets
                    bowlers = self.deliveries[self.deliveries['bowling_team'] == team].bowler.unique()
                    for bowler in bowlers:
                        total_wickets = self.deliveries[self.deliveries['bowler'] == bowler].dismissal_kind.count()
                        run_outs = self.deliveries[(self.deliveries['bowler'] == bowler) & (self.deliveries['dismissal_kind']== 'run out')].dismissal_kind.count()
                        wickets = total_wickets - run_outs
                        bowler_wicket.append((bowler, wickets, team, wickets_total))

                elif (bowler == None): 
                    bowlers = self.deliveries.bowler.unique()
                    for bowler in bowlers:
                        total_wickets = self.deliveries[self.deliveries['bowler'] == bowler].dismissal_kind.count()
                        run_outs = self.deliveries[(self.deliveries['bowler'] == bowler) & (self.deliveries['dismissal_kind']== 'run out')].dismissal_kind.count()
                        wickets = total_wickets - run_outs
                        bowler_wicket.append((bowler, wickets))
                else:
                    total_wickets = self.deliveries[self.deliveries['bowler'] == bowler].dismissal_kind.count()
                    run_outs = self.deliveries[(self.deliveries['bowler'] == bowler) & (self.deliveries['dismissal_kind']== 'run out')].dismissal_kind.count()
                    wickets = total_wickets - run_outs
                    bowler_wicket.append((bowler, wickets))
            else:
                if (team is not None):
                    total_wickets = self.deliveries[(self.deliveries['match_id'] == match_id) & (self.deliveries['bowling_team'] == team)].dismissal_kind.count()
                    wickets_total = total_wickets 
                    bowlers = self.deliveries[(self.deliveries['match_id'] == match_id)& (self.deliveries['bowling_team'] == team)].bowler.unique()
                    for bowler in bowlers:
                        total_wickets = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].dismissal_kind.count()
                        run_outs = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler) & (self.deliveries['dismissal_kind']== 'run out')].dismissal_kind.count()
                        wickets = total_wickets - run_outs
                        bowler_wicket.append((bowler, wickets, team, wickets_total))
                elif (bowler == None):
                    bowlers = self.deliveries[(self.deliveries['match_id'] == match_id)].bowler.unique()
                    for bowler in bowlers:
                        total_wickets = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].dismissal_kind.count()
                        run_outs = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler) & (self.deliveries['dismissal_kind']== 'run out')].dismissal_kind.count()
                        wickets = total_wickets - run_outs
                        bowler_wicket.append((bowler, wickets))
                else:
                    total_wickets = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler)].dismissal_kind.count()
                    run_outs = self.deliveries[(self.deliveries['match_id'] == match_id)&(self.deliveries['bowler'] == bowler) & (self.deliveries['dismissal_kind']== 'run out')].dismissal_kind.count()
                    wickets = total_wickets - run_outs
                    bowler_wicket.append((bowler, wickets))
            bowler_wicket = sorted(bowler_wicket, key=lambda x: x[1], reverse=True)
            return  bowler_wicket[rank-1]

In [4]:
# Loading Training Sentences
with open('tagged_training_sentences.txt') as tagged_sentence:
    corrected_train = tagged_sentence.read()

In [5]:
class POSTagger(SequentialBackoffTagger):
    def __init__(self, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)
    
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        return nltk.pos_tag([word])[0][1] if word != "" else None

In [6]:
custom_pos_tagger = POSTagger()

In [7]:
def transform_str2tuple(tagged_sentence):
    tagged_sentence_tuple = []
    sentences = tagged_sentence.split("\n")
    for sentence in sentences:
        tagged_question = []
        for word in word_tokenize(sentence):
            tagged_question.append(nltk.str2tuple(word))
        tagged_sentence_tuple.append(tagged_question)
    return tagged_sentence_tuple

In [8]:
def get_brill_tagger(tagged_sentences):
    templates = [brill.Template(brill.Pos([1,1])),
    brill.Template(brill.Pos([2,2])),
    brill.Template(brill.Pos([1,2])),
    brill.Template(brill.Pos([1,3])),
    brill.Template(brill.Pos([1,1])),
    brill.Template(brill.Pos([2,2])),
    brill.Template(brill.Pos([1,2])),
    brill.Template(brill.Pos([1,3])),
    brill.Template(brill.Word([-1, -1])),
    brill.Template(brill.Word([-1, -1]))]

    trainer_initial_pos = BrillTaggerTrainer(initial_tagger= custom_pos_tagger, templates=templates, trace=3, deterministic=True)
    brill_tagger = trainer_initial_pos.train(tagged_sentences, max_rules=10)
    
    return brill_tagger

In [9]:
brill_tagger = get_brill_tagger(transform_str2tuple(corrected_train))

TBL train (fast) (seqs: 138; tokens: 1451; tpls: 10; min score: 2; min acc: None)
Finding initial useful rules...
    Found 710 useful rules.

           B      |
   S   F   r   O  |        Score = Fixed - Broken
   c   i   o   t  |  R     Fixed = num tags changed incorrect -> correct
   o   x   k   h  |  u     Broken = num tags changed correct -> incorrect
   r   e   e   e  |  l     Other = num tags changed incorrect -> incorrect
   e   d   n   r  |  e
------------------+-------------------------------------------------------
 116 116   0   0  | .->None if Pos:None@[1]
 116 116   0   0  | None-> if Pos:.@[1]
 116 116   0   0  | .->None if Word:@[-1]
  34  42   8   9  | NN->NNP if Pos:NN@[1]
  15  18   3   8  | NN->NNP if Pos:IN@[2]
  14  14   0   0  | NN->NNP if Pos:POS@[1,2]
  14  14   0   0  | VBN->VBD if Word:who@[-1]
   8  11   3   0  | NN->NNP if Word:by@[-1]
   6   6   0   2  | NNP->VBZ if Pos:CD@[1]
   5   7   2   0  | NNS->NNP if Pos:NNP@[1]


In [10]:
train_sentences = pd.read_csv("training_sentences_classifier.csv")

In [100]:
class FeatureExtractor:
    def __init__(self):
        with open('features.json') as features:
            self.features = json.load(features)
            
    def extract_features(self, tokenized_words):
        features = self.features
        #tokenized_sentence = nltk.word_tokenize(sentence)
        word_counts = nltk.Counter(tokenized_words)
        for word in word_counts:
            if word in features:
                features[word] = True
        for feature in features:
            if features[feature] == 0:
                features[feature] = False
        return features
    
    def extract_features_rf(self, tokenized_words):
        features = self.features
        one_hot_encoded = []
        #tokenized_sentence = nltk.word_tokenize(sentence)
        word_counts = nltk.Counter(tokenized_words)
        for word in word_counts:
            if word in features:
                features[word] = True

        for feature in features:
            one_hot_encoded.append(bool(features[feature]))
        return one_hot_encoded

In [102]:
fe = FeatureExtractor()

In [87]:
[sentence for index, sentence, label in train_sentences.values]

['Total runs scored by SC Ganguly in match 5?',
 'SC Ganguly score in match 1?',
 'how many runs did Ganguly score in match 2?',
 "Sachin's score in 4th match?",
 'how much did McCullum scored in match 3?',
 'how much did McCullum scored in match 4?',
 'McCullum runs in match 3?',
 "Dravid's runs in match 2?",
 "Dravid's score in match 5?",
 'How much did RCB Score in Match 1?',
 'RCB Score in match 1?',
 'RCB runs in match 1',
 'total score of DD in 3rd match',
 'totals runs of Deccan in 8th match',
 'final score of DD in 3rd match',
 'Rahul maximum score in match 3',
 'who was the leading run scorer in match 9',
 'highest score in match 3',
 'maximum scorer of match 2',
 'who scored the maximum run in match 3 from DD',
 'top scorer for DD in match 3',
 'top score in DD match 4?',
 'highest scorer for DD in match 4',
 'who scored maximum runs in match 6',
 'who was the top scorer in match 6',
 'who was the leading run scorer in match 9',
 'who scored maximum runs in match 6',
 'who wa

In [88]:
naive_bayes_classifier = nltk.classify.naivebayes.NaiveBayesClassifier.train([(fe.extract_features(nltk.word_tokenize(sentence)), label) for index, sentence, label in train_sentences.values])

In [93]:
pc = naive_bayes_classifier.classify(fe.extract_features(word_tokenize("runs scored by sourav ganguly")))

In [94]:
pc

'strike_rate'